In [1]:
# Install pymongo (jika belum terpasang)
!pip install pymongo

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 7.4 MB/s eta 0:00:00 MB/s eta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [pymongo]━━━ 1/2 [pymongo]


In [3]:
from pymongo import MongoClient

# Inisialisasi client dan koneksi ke database
client = MongoClient('mongodb://localhost:27017/')
db = client['company_db']
collection = db['employees']

# Contoh untuk memastikan koneksi
print('Connected to MongoDB')


Connected to MongoDB


In [4]:
# --- Tugas: CRUD di MongoDB ---

# 1️⃣ Insert satu data
employee_data = {
    'name': 'Alice',
    'department': 'Finance',
    'age': 29,
    'salary': 4500
}
collection.insert_one(employee_data)
print('1 Data inserted (Alice)')

# 2️⃣ Insert beberapa data tambahan
more_employees = [
    {'name': 'Bob', 'department': 'IT', 'age': 31, 'salary': 5000},
    {'name': 'Charlie', 'department': 'HR', 'age': 26, 'salary': 3800},
    {'name': 'Diana', 'department': 'Marketing', 'age': 27, 'salary': 4200}
]
collection.insert_many(more_employees)
print('Multiple data inserted!')

# 3️⃣ Menampilkan semua data
print("\n=== All Employees ===")
for doc in collection.find():
    print(doc)

# 4️⃣ Update data (contoh: ubah gaji Charlie)
collection.update_one(
    {'name': 'Charlie'},             # kondisi data yang mau diupdate
    {'$set': {'salary': 4000}}       # nilai baru
)
print("\nUpdated Charlie's salary!")

# 5️⃣ Delete data (contoh: hapus data Diana)
collection.delete_one({'name': 'Diana'})
print("Deleted Diana's record!")

# 6️⃣ Tampilkan kembali data setelah update & delete
print("\n=== Final Employee Data ===")
for doc in collection.find():
    print(doc)


1 Data inserted (Alice)
Multiple data inserted!

=== All Employees ===
{'_id': ObjectId('6902e49b021a6de543b4c59e'), 'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 4500}
{'_id': ObjectId('6902e49b021a6de543b4c59f'), 'name': 'Bob', 'department': 'IT', 'age': 31, 'salary': 5000}
{'_id': ObjectId('6902e49b021a6de543b4c5a0'), 'name': 'Charlie', 'department': 'HR', 'age': 26, 'salary': 3800}
{'_id': ObjectId('6902e49b021a6de543b4c5a1'), 'name': 'Diana', 'department': 'Marketing', 'age': 27, 'salary': 4200}

Updated Charlie's salary!
Deleted Diana's record!

=== Final Employee Data ===
{'_id': ObjectId('6902e49b021a6de543b4c59e'), 'name': 'Alice', 'department': 'Finance', 'age': 29, 'salary': 4500}
{'_id': ObjectId('6902e49b021a6de543b4c59f'), 'name': 'Bob', 'department': 'IT', 'age': 31, 'salary': 5000}
{'_id': ObjectId('6902e49b021a6de543b4c5a0'), 'name': 'Charlie', 'department': 'HR', 'age': 26, 'salary': 4000}


In [5]:
# Query Aggregation untuk mencari rata-rata gaji per departemen
pipeline = [
    {
        '$group': {
            '_id': '$department',              # Mengelompokkan berdasarkan kolom 'department'
            'average_salary': {'$avg': '$salary'}  # Menghitung rata-rata gaji tiap kelompok
        }
    }
]

print("=== Average Salary per Department ===")
for result in collection.aggregate(pipeline):
    print(f"Department: {result['_id']}, Average Salary: {result['average_salary']:.2f}")


=== Average Salary per Department ===
Department: Finance, Average Salary: 4500.00
Department: IT, Average Salary: 5000.00
Department: HR, Average Salary: 4000.00


In [6]:
#latihan 4.1
# Buat koleksi baru bernama 'products'
products = db['products']

# Tambahkan minimal 10 data produk
product_data = [
    {'name': 'Laptop', 'category': 'Electronics', 'price': 8500},
    {'name': 'Mouse', 'category': 'Electronics', 'price': 250},
    {'name': 'Keyboard', 'category': 'Electronics', 'price': 400},
    {'name': 'Monitor', 'category': 'Electronics', 'price': 2000},
    {'name': 'Chair', 'category': 'Furniture', 'price': 1200},
    {'name': 'Table', 'category': 'Furniture', 'price': 1500},
    {'name': 'Sofa', 'category': 'Furniture', 'price': 3500},
    {'name': 'Notebook', 'category': 'Stationery', 'price': 50},
    {'name': 'Pen', 'category': 'Stationery', 'price': 30},
    {'name': 'Ruler', 'category': 'Stationery', 'price': 20}
]

# Masukkan ke koleksi
products.insert_many(product_data)
print("✅ 10 produk berhasil dimasukkan ke koleksi 'products'")


✅ 10 produk berhasil dimasukkan ke koleksi 'products'


In [7]:
#latihan 4.2
# Hitung rata-rata harga
pipeline_avg = [
    {'$group': {'_id': None, 'avg_price': {'$avg': '$price'}}}
]
avg_result = list(products.aggregate(pipeline_avg))[0]['avg_price']
print(f"💰 Rata-rata harga produk: {avg_result:.2f}\n")

# Cari produk dengan harga di atas rata-rata
print("=== Produk dengan harga di atas rata-rata ===")
for p in products.find({'price': {'$gt': avg_result}}):
    print(f"{p['name']} - {p['category']} - Rp{p['price']}")


💰 Rata-rata harga produk: 1745.00

=== Produk dengan harga di atas rata-rata ===
Laptop - Electronics - Rp8500
Monitor - Electronics - Rp2000
Sofa - Furniture - Rp3500


In [8]:
#latihan 4.3
pipeline_count = [
    {'$group': {'_id': '$category', 'total_products': {'$sum': 1}}}
]

print("\n=== Total Produk per Kategori ===")
for result in products.aggregate(pipeline_count):
    print(f"Kategori: {result['_id']}, Jumlah Produk: {result['total_products']}")



=== Total Produk per Kategori ===
Kategori: Furniture, Jumlah Produk: 3
Kategori: Electronics, Jumlah Produk: 4
Kategori: Stationery, Jumlah Produk: 3


In [9]:
#tugas 1
pipeline_top5 = [
    {'$sort': {'department': 1, 'salary': -1}},  # Urutkan per departemen dan gaji tertinggi
    {'$group': {
        '_id': '$department',
        'top5_employees': {'$push': {'name': '$name', 'salary': '$salary'}},
    }},
    {'$project': {
        '_id': 1,
        'top5_employees': {'$slice': ['$top5_employees', 5]}  # Ambil hanya 5 teratas
    }}
]

print("=== 5 Karyawan dengan Gaji Tertinggi per Departemen ===")
for dept in collection.aggregate(pipeline_top5):
    print(f"\nDepartemen: {dept['_id']}")
    for emp in dept['top5_employees']:
        print(f"- {emp['name']} | Gaji: {emp['salary']}")


=== 5 Karyawan dengan Gaji Tertinggi per Departemen ===

Departemen: Finance
- Alice | Gaji: 4500

Departemen: IT
- Bob | Gaji: 5000

Departemen: HR
- Charlie | Gaji: 4000


In [10]:
#tugas 2
result = collection.delete_many({'age': {'$lt': 25}})
print(f"\n🗑️ {result.deleted_count} karyawan berusia di bawah 25 tahun telah dihapus.")


🗑️ 0 karyawan berusia di bawah 25 tahun telah dihapus.


In [11]:
#tugas 3
pipeline_report = [
    {'$group': {
        '_id': '$department',
        'total_salary': {'$sum': '$salary'},
        'average_age': {'$avg': '$age'}
    }}
]

print("\n=== Laporan Total Gaji & Rata-rata Umur per Departemen ===")
for dept in collection.aggregate(pipeline_report):
    print(f"Departemen: {dept['_id']}")
    print(f"  Total Gaji : {dept['total_salary']}")
    print(f"  Rata-rata Umur : {dept['average_age']:.2f}\n")



=== Laporan Total Gaji & Rata-rata Umur per Departemen ===
Departemen: HR
  Total Gaji : 4000
  Rata-rata Umur : 26.00

Departemen: IT
  Total Gaji : 5000
  Rata-rata Umur : 31.00

Departemen: Finance
  Total Gaji : 4500
  Rata-rata Umur : 29.00

